In [ ]:
!pip install -q ../input/faiss-163/faiss_gpu-1.6.3-cp37-cp37m-manylinux2010_x86_64.whl

In [ ]:
import os
import gc
import cv2
import math
import copy
import time
import random

import cudf
import cupy
import pandas as pd
import numpy as np
import xgboost as xgb
import numpy as np
import pandas as pd
from tqdm import tqdm

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# For Transformer Models
import transformers
from transformers import AutoTokenizer, AutoModel, AdamW, AutoConfig

from cuml.neighbors import NearestNeighbors
from cuml.feature_extraction.text import TfidfVectorizer
from cuml.metrics import pairwise_distances

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold

from unidecode import unidecode


class CFG:
    seed = 42 # 随机种子
    n_candidates = 50 # 候选集数量
    n_spatial_candidates = 10 # 空间候选集数量
    n_splits = 4 # 折数
    max_length = 64 # 最大长度

    device = torch.device('cuda') # GPU

    # Arcface
    s = 30.0 # arcface 参数 scale
    m = 0.5  # arcface 参数 margin
    ls_eps = 0.0 # arcface 参数 eps
    easy_margin = False # easy_margin
    n_classes = 739972 # 分类数量

    
# 四个模型
class CFG_Model1:
    model_name = "../input/huggingface-roberta-variants/xlm-roberta-large/xlm-roberta-large" 
    weight = "../input/model1/xlm-roberta-large_epoch30.bin"
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

class CFG_Model2:
    model_name = "../input/sentence-transformers/LaBSE/0_Transformer"
    weight = "../input/model2/0_Transformer_epoch30.bin"
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    
class CFG_Model3:
    model_name = "../input/paraphrasemultilingualmpnetbasev2/paraphrase-multilingual-mpnet-base-v2"
    weight = "../input/model3/paraphrase-multilingual-mpnet-base-v2_epoch30.bin"
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    
class CFG_Model4:
    model_name = "../input/rembert-pt"
    weight = "../input/model4/rembert-pt_epoch30.bin"
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

In [ ]:
# test.csv
df = cudf.read_csv("../input/foursquare-location-matching/test.csv", dtype={
    "id": str, "name": str, "latitude": float, "longitude": float, "address": str, 
    "city": str, "state": str, "zip": str, "country": str, "url": str, "phone": str, "categories": str
})

# train.csv
df_train = cudf.read_csv("../input/foursquare-location-matching/train.csv", dtype={
    "id": str, "name": str, "latitude": float, "longitude": float, "address": str, 
    "city": str, "state": str, "zip": str, "country": str, "url": str, "phone": str, "categories": str, "point_of_interest": str
}).drop(columns=["point_of_interest"])

# 全量数据
df_all = cudf.concat([df, df_train], axis=0).reset_index(drop=True)

In [ ]:
from sklearn.neighbors import BallTree

# Creates new columns converting coordinate degrees to radians.
for column in ["latitude", "longitude"]:
    # 将经度或纬度 --> 弧度
    rad = np.deg2rad(df_all[column].to_pandas().values) # df_all
    df_all[f'{column}_rad'] = rad 
    rad = np.deg2rad(df[column].to_pandas().values) # df
    df[f'{column}_rad'] = rad


k = 11
ball = BallTree(df_all[["latitude_rad", "longitude_rad"]].to_pandas().values, metric='haversine') # 创建BallTree
D_latlon, I_latlon = ball.query(df[["latitude_rad", "longitude_rad"]].to_pandas().values, k=k)  # 计算每个点到其他点的距离

In [ ]:
fill_columns = ["city", "state", "country"] # 填充的列
print("Before fillna") # 打印前几行
print(df[fill_columns].isnull().sum() / df.shape[0]) # 打印缺失值比例
for i in range(1, 6):  # 使用前5个candidate
    nearest_index = I_latlon[:, i] # 取出第i个candidate
    for c in fill_columns:
        df[c] = df[c].fillna(df_all.loc[nearest_index, c].reset_index(drop=True)) # 对空值填充第i个candidate的"city", "state", "country"

In [ ]:
for col in ["name", "address", "city", "state", "zip", "country", "url", "phone", "categories"]:
    df[col] = df[col].fillna("") # 对空值填充空字符串

# fulltext = 拼接name、address、city、state、country、categories
df["fulltext"] = (
    df["name"] + " " + df["address"] + " " + df["city"] + " " + df["state"] + " "  + df["country"] + " " + df["categories"]
).to_pandas().replace(r'\s+', ' ', regex=True) # 

# Standardization of coordinates.
# https://datascience.stackexchange.com/questions/13567/ways-to-deal-with-longitude-latitude-feature
df["coord_x"] = cupy.cos(df["latitude"]) * cupy.cos(df["longitude"]) # 经度和纬度转换成x坐标
df["coord_y"] = cupy.cos(df["latitude"]) * cupy.sin(df["longitude"]) # 经度和纬度转换成y坐标
df["coord_z"] = cupy.sin(df["latitude"]) # 经度和纬度转换成z坐标
                       
print(df.shape)
df.head()

In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(CFG.seed)

## Load ArcFace model

In [ ]:
class FourSquareDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.fulltext = df['fulltext'].values # 全文
        self.latitudes = df['latitude'].values # 纬度
        self.longitudes = df['longitude'].values # 经度
        self.coord_x = df['coord_x'].values # 经纬度坐标 x
        self.coord_y = df['coord_y'].values # 经纬度坐标 y
        self.coord_z = df['coord_z'].values # 经纬度坐标 z
        self.tokenizer = tokenizer # 词表
        self.max_length = max_length # 最大长度
        
    def __len__(self):
        return len(self.fulltext) # 返回数据长度
    
    def __getitem__(self, index):
        fulltext = self.fulltext[index] # 全文
        latitude = self.latitudes[index] # 纬度
        longitude = self.longitudes[index] # 经度
        coord_x = self.coord_x[index] # 经纬度坐标 x
        coord_y = self.coord_y[index] # 经纬度坐标 y
        coord_z = self.coord_z[index] # 经纬度坐标 z
        
        inputs = self.tokenizer(
            fulltext, # 全文
            truncation=True, # 截断
            add_special_tokens=True, # 添加特殊字符
            max_length=self.max_length, # 最大长度
            padding='max_length', # 填充方式
            return_tensors="pt" # 返回张量
        )

        return {
            'ids': inputs['input_ids'][0], # 输入张量
            'mask': inputs['attention_mask'][0], # 注意力掩码
            'latitude': torch.tensor(latitude, dtype=torch.float), # 纬度
            'longitude': torch.tensor(longitude, dtype=torch.float), # 经度
            'coord_x': torch.tensor(coord_x), # 经纬度坐标 x
            'coord_y': torch.tensor(coord_y), # 经纬度坐标 y
            'coord_z': torch.tensor(coord_z), # 经纬度坐标 z
        }

In [ ]:
# Arcface
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, 
                 m=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features  # input的维度
        self.out_features = out_features # output的维度
        self.s = s # re-scale
        self.m = m # margin
        self.ls_eps = ls_eps  # label smoothing
        # 初始化权重
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin # easy_margin 模式
        self.cos_m = math.cos(m) # cos margin
        self.sin_m = math.sin(m) # sin margin
        self.threshold = math.cos(math.pi - m) # cos(pi - m) = -cos(m)
        self.mm = math.sin(math.pi - m) * m # sin(pi - m)*m = sin(m)*m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight)) # 获得cosθ (vector)
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2)) # 获得cosθ
        phi = cosine * self.cos_m - sine * self.sin_m # cosθ*cosm – sinθ*sinm = cos(θ + m)
        phi = phi.float() # phi to float
        cosine = cosine.float() # cosine to float
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            # 以下代码控制了 θ+m 应该在 range[0, pi]
            # if cos(θ) > cos(pi - m) means θ + m < math.pi, so phi = cos(θ + m);
            # else means θ + m >= math.pi, we use Talyer extension to approximate the cos(θ + m).
            # if fact, cos(θ + m) = cos(θ) - m * sin(θ) >= cos(θ) - m * sin(math.pi - m)
            phi = torch.where(cosine > self.threshold, phi, cosine - self.mm) # https://github.com/ronghuaiyang/arcface-pytorch/issues/48
        # --------------------------- convert label to one-hot ---------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        # 对label形式进行转换，假设batch为2、有3类的话，即将label从[1,2]转换成[[0,1,0],[0,0,1]]
        one_hot = torch.zeros(cosine.size(), device=CFG.device)
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) ------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        # 进行re-scale
        output *= self.s

        return output

class FSMultiModalNet(nn.Module):
    def __init__(self, model_name, fc_dim, num_features=3):
        super(FSMultiModalNet, self).__init__()
        self.config = AutoConfig.from_pretrained(model_name) # 加载预训练模型
        self.bert_model = AutoModel.from_pretrained(model_name, config=self.config) # 加载预训练模型
        # self.embedding = nn.Linear(self.config.hidden_size + 2, embedding_size)

        self.fc = nn.Linear(self.bert_model.config.hidden_size + num_features, fc_dim) # 全连接层 hidden_size + x'y'z'
        self.bn = nn.BatchNorm1d(fc_dim) # BatchNorm1d
        self._init_params() # 初始化参数

        self.margin = ArcMarginProduct(
            fc_dim, # 输入维度
            CFG.n_classes, # 输出维度
            s=CFG.s, # scale
            m=CFG.m, # margin 
            easy_margin=CFG.easy_margin, # easy_margin
            ls_eps=CFG.ls_eps # label smoothing epsilon
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight) # 初始化全连接层权重
        nn.init.constant_(self.fc.bias, 0) # 初始化全连接层偏置
        nn.init.constant_(self.bn.weight, 1) # 初始化 BatchNorm1d 权重
        nn.init.constant_(self.bn.bias, 0) # 初始化 BatchNorm1d 偏置

    def forward(self, ids, mask, lat, lon, coord_x, coord_y, coord_z, labels):
        feature = self.extract_feature(ids, mask, lat, lon, coord_x, coord_y, coord_z) # 提取特征
        output = self.margin(feature, labels) # ArcMarginProduct 输出

        return output
    
    def extract_feature(self, input_ids, attention_mask, lat, lon, coord_x, coord_y, coord_z):
        x = self.bert_model(input_ids=input_ids, attention_mask=attention_mask) # 获取 BERT 特征
        x = torch.sum(x.last_hidden_state * attention_mask.unsqueeze(-1), dim=1) / attention_mask.sum(dim=1, keepdims=True) # 将 BERT 特征attention_mask部分求平均

        x = torch.cat([x, coord_x.view(-1, 1), coord_y.view(-1, 1), coord_z.view(-1, 1)], axis=1) # 将 bert输出 和 x'y'z' 合并

        x = self.fc(x) # 全连接层
        x = self.bn(x) # BatchNorm1d

        return x # 返回特征

In [ ]:
def get_embed(model,tokenizer):
    # NN embeddings 
    dataset = FourSquareDataset(df.to_pandas(), tokenizer=tokenizer, max_length=CFG.max_length) # Dataset
    loader = DataLoader(dataset, batch_size=512, num_workers=6, shuffle=False, pin_memory=True) # DataLoader

    embeds = []
    with torch.no_grad():
        for data in tqdm(loader): 
            ids = data['ids'].to(CFG.device, dtype=torch.long) # input_ids
            mask = data['mask'].to(CFG.device, dtype=torch.long) # attention_mask

            latitude = data['latitude'].to(CFG.device, dtype=torch.float) # latitude
            longitude = data['longitude'].to(CFG.device, dtype=torch.float) # longitude
            coord_x = data['coord_x'].to(CFG.device, dtype=torch.float) # coord_x
            coord_y = data['coord_y'].to(CFG.device, dtype=torch.float) # coord_y
            coord_z = data['coord_z'].to(CFG.device, dtype=torch.float) # coord_z
            # labels = data['label'].to(CFG.device, dtype=torch.long)

            emb = model.extract_feature(ids, mask, latitude, longitude, coord_x, coord_y, coord_z) # embeddings
            embeds.append(emb.detach().cpu().numpy()) # embeddings list

    V_embed_bert = cupy.array(np.concatenate(embeds)) # concatenate embeddings --> array
    V_embed_bert = V_embed_bert / cupy.linalg.norm(V_embed_bert, ord=2, axis=1, keepdims=True) # l2 normalization
    return V_embed_bert # 返回 embeddings


def get_embed_model(model_name,fcdim, state_dict,tokenizer):
    model = FSMultiModalNet(model_name, fcdim) # 初始化模型
    model.to(CFG.device); # 将模型转到GPU
    model.load_state_dict(torch.load(state_dict)) # 加载模型参数
    embed = get_embed(model,tokenizer) # 获取embedding

    del model
    gc.collect()
    torch.cuda.empty_cache()
    return embed

In [ ]:
# Embedding1
embed1 = get_embed_model(CFG_Model1.model_name, 320, CFG_Model1.weight, CFG_Model1.tokenizer)
gc.collect()
torch.cuda.empty_cache()

# Embedding2
embed2 = get_embed_model(CFG_Model2.model_name, 320, CFG_Model2.weight, CFG_Model2.tokenizer)
gc.collect()
torch.cuda.empty_cache()

# Embedding3
embed3 = get_embed_model(CFG_Model3.model_name, 320, CFG_Model3.weight, CFG_Model3.tokenizer)
gc.collect()
torch.cuda.empty_cache()

# Embedding4
embed4 = get_embed_model(CFG_Model4.model_name, 320, CFG_Model4.weight, CFG_Model4.tokenizer)
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# l2 normalization and concat
V_embed_concat = cupy.concatenate([
    embed1, # Embedding1
    embed2, # Embedding2
    embed3, # Embedding3
    embed4, # Embedding4
], axis=1)

del embed1,embed2,embed3,embed4
gc.collect()
torch.cuda.empty_cache()

V_embed_concat = V_embed_concat / cupy.linalg.norm(V_embed_concat, ord=2, axis=1, keepdims=True) # l2 normalization

gc.collect()
torch.cuda.empty_cache()

## Generate candidate rank

In [ ]:
# Create candidate index country by country
import faiss

def gen_candidate_ranks(df, V_embed, n_candidates, no_country=False):
    '''
    Generate candidate ranks for each row in df.
    '''
    res = faiss.StandardGpuResources() # GPU资源
    index = faiss.IndexFlatIP(V_embed.shape[1])  # 创建索引
    index = faiss.index_cpu_to_gpu(res, 0, index) # 将索引放到GPU上
    index.add(V_embed) # 将embedding加入索引
    D, I = index.search(V_embed, CFG.n_candidates) # 搜索最近的n_candidates个索引
    I = np.where(I == -1, 0, I) # 将-1替换为0
    D = np.clip(D, 0, 1) # 将D值限制在0和1之间
    return D, I

In [ ]:
# 寻找每个样本的n个最近邻值的距离和索引
D_concat, I_concat = gen_candidate_ranks(df, cupy.asnumpy(V_embed_concat), CFG.n_candidates)

gc.collect()
torch.cuda.empty_cache()

# DBA/QE weighted

In [ ]:
# https://www.kaggle.com/code/lyakaap/2nd-place-solution/notebook
def query_expansion(V, D, I, alpha=3, k=2):
    '''
    对V_embed_concat进行查询扩展. 
    当前样本的embedding, 用最邻近的2个embedding乘以距离权重, 然后求和.

    V: embedding matrix
    D: distance matrix
    I: index matrix
    alpha: 次方系数
    k: Top-k 邻近值
    '''
    weights = cupy.array(np.expand_dims(D[:, :k] ** alpha, axis=-1).astype(np.float32)) # 每个样本前k个邻近距离的3次方， shape==(1138812, 2, 1) 
    chunk_size = 100_000 # chunk大小, 防止OOM
    for i in range(0, len(df), chunk_size): 
        # V.shape: (1138812, 960)
        # I[i:i+chunk_size, :k].shape: (100000, 2) # 纯作为索引
        # V[I[i:i+chunk_size, :k]].shape: (100000, 2, 960)
        # weights[i:i+chunk_size].shape: (100000, 2, 1)
        V[i:i+chunk_size] = (V[I[i:i+chunk_size, :k]] * weights[i:i+chunk_size]).sum(axis=1) # 将V中的2个embedding乘上，以距离为权重，然后求和
    return V


V_embed_concat = query_expansion(V_embed_concat, D_concat, I_concat) # 对V_embed_concat进行查询扩展. 当前样本的embedding, 用最邻近的2个embedding乘以距离权重, 然后求和.
V_embed_concat /= np.linalg.norm(V_embed_concat, 2, axis=1, keepdims=True) # l2 正则化

del D_concat,I_concat # 删除变量
gc.collect() # 清空缓存
torch.cuda.empty_cache() # 清空显存

D_concat, I_concat = gen_candidate_ranks(df, cupy.asnumpy(V_embed_concat), CFG.n_candidates) # 对V_embed_concat进行索引

gc.collect() # 清空缓存
torch.cuda.empty_cache() # 清空显存

In [ ]:
from sklearn.neighbors import BallTree

def gen_candidate_rank_spatial(df, n_candidates):
    '''
    对每个country内的样本进行k邻近查找
    按经度/纬度的haversine距离查找candidates。
    Returns:
        I (np.array): I[s_index][r] -> s_index 的第r个最近的点的索引。
    '''
    for column in df[["latitude", "longitude"]]:
        rad = np.deg2rad(df[column].values) # 将经度或纬度 --> 弧度
        df[f'{column}_rad'] = rad # 弧度 存入df

    I = np.full((len(df), n_candidates), -1, dtype=np.int32) # 创建I矩阵, shape=(sample_number, n_candidates), 全为-1
    for country, country_df in tqdm(df[["country", "latitude_rad", "longitude_rad"]].to_pandas().groupby("country")): # 对每个国家进行遍历
        clip_n_candidates = min(len(country_df), n_candidates) # 最大只允许n_candidates个样本
        country_df = country_df.reset_index() # 重置索引
        ball = BallTree(country_df[["latitude_rad", "longitude_rad"]].values, metric='haversine') # 创建BallTree
        # 计算country_df中每个点到其他点的距离
        _, indices = ball.query( 
            country_df[["latitude_rad", "longitude_rad"]].values,  # 待查询点的经纬度
            k = clip_n_candidates # k个最邻近点
        )
        
        indices = np.concatenate(
            [
                indices,  # BallTree查询结果索引
                np.zeros((len(indices), n_candidates - clip_n_candidates), dtype=np.int32) # 当candidates列数过少时，用0补全至n_candidates列数
            ], axis=1
        )
        for i in range(n_candidates):
            I[country_df["index"].values, i] = country_df.loc[indices[:, i], "index"].values

    return I

I_spatial = gen_candidate_rank_spatial(df, 10) # 对每个country内的样本进行k邻近查找 shpe==(1138812, 10)

## tfidf

In [ ]:
from cuml.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer() # 创建tfidf
V_name = tfidf.fit_transform(df["name"])  # 对name进行tfidf
V_name.shape

In [ ]:
tfidf = TfidfVectorizer(stop_words='english') # 创建tfidf
df["full_address"] = df["address"] + ", " + df["city"] + ", " + df["state"] + ", "  + df["country"] # 创建full_address列, 为 address, city, state, country
V_full_address = tfidf.fit_transform(df["full_address"]) # 对full_address进行tfidf
V_full_address.shape

In [ ]:
tfidf = TfidfVectorizer() # 创建tfidf
V_cat = tfidf.fit_transform(df["categories"]) # 对categories进行tfidf
V_cat.shape

## Predict

In [ ]:
from cuml import ForestInference

THRESHOLD = 0.3 # 阈值

In [ ]:
R = 6371.0  # 地球半径, 单位km

def manhattan(lat1, long1, lat2, long2):
    '''
    计算曼哈顿距离
    '''
    return np.abs(lat2 - lat1) + np.abs(long2 - long1)


def haversine_np(lon1, lat1, lon2, lat2):
    """
    https://www.kaggle.com/code/justfor/speedup-haversine/script
    计算地球上两点之间的大圆距离的距离（以小数点后的度数指定）。
    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1 # 经度差
    dlat = lat2 - lat1 # 纬度差

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = R * c
    return km


def create_features(df, i, indices):
    '''
    创建特征
    '''
    prev_i = max(i-1, 0) # i-1,最小值为0
    next_i = min(i+1, indices.shape[1] - 1) # i+1,最大值为indices长度
    
    prev_cand_index = indices[:, prev_i] # 第i-1个相似度的candidate的索引
    next_cand_index = indices[:, next_i] # 第i+1个相似度的candidate的索引
    cand_index = indices[:, i] # 第i个相似度的candidate索引
    
    lon1 = df["longitude"].to_pandas().to_numpy() # longitude列的值
    lat1 = df["latitude"].to_pandas().to_numpy() # latitude列的值
    lon2 = df["longitude"][cand_index].to_pandas().to_numpy() # 第i个相似度的candidate顺序的longitude列值
    lat2 = df["latitude"][cand_index].to_pandas().to_numpy() # 第i个相似度的candidate顺序的latitude列值
    #df["diff_lon"] = lon1 - lon2
    #df["diff_lat"] = lat1 - lat2
    df["diff_lon"] = np.abs(lon1 - lon2) # 经度差绝对值
    df["diff_lat"] = np.abs(lat1 - lat2) # 纬度差绝对值
    
    df["lonlat_eucdist"] =  (df['diff_lon'] ** 2 + df['diff_lat'] ** 2) ** 0.5 # 经纬度差的平方根
    df["lonlat_manhattan"] = manhattan(lat1, lon1, lat2, lon2) # 曼哈顿距离
    df["lonlat_haversine_dist"] = haversine_np(lon1, lat1, lon2, lat2) # 地球距离
    
    df["name_cossim"] = V_name.multiply(V_name[cand_index]).sum(axis=1).ravel() # 名称相似度
    df["full_address_cossim"] = V_full_address.multiply(V_full_address[cand_index]).sum(axis=1).ravel() # 地址相似度
    df["cat_cossim"] = V_cat.multiply(V_cat[cand_index]).sum(axis=1).ravel() # 类别相似度
    
    df["cand_hit_count_02"] = df["hit_count_02"][cand_index].to_pandas().to_numpy()   # 预测概率>0.2的样本的candidate的概率
    df["cand_hit_count_03"] = df["hit_count_03"][cand_index].to_pandas().to_numpy()   # 预测概率>0.3的样本的candidate的概率
    df["cand_hit_count_04"] = df["hit_count_04"][cand_index].to_pandas().to_numpy()   # 预测概率>0.4的样本的candidate的概率 
    df["cand_hit_count_05"] = df["hit_count_05"][cand_index].to_pandas().to_numpy()   # 预测概率>0.5的样本的candidate的概率
    df["cand_hit_count_sum"] = df["hit_count_sum"][cand_index].to_pandas().to_numpy() # 所有样本的candidate的概率
    
    df["hit_count_02_min"] = df[["hit_count_02", "cand_hit_count_02"]].min(axis=1)  # 样本预测概率和其candidate的概率 的最小值（0.2）
    df["hit_count_03_min"] = df[["hit_count_03", "cand_hit_count_03"]].min(axis=1) # 样本预测概率和其candidate的概率 的最小值（0.3）
    df["hit_count_04_min"] = df[["hit_count_04", "cand_hit_count_04"]].min(axis=1) # 样本预测概率和其candidate的概率 的最小值（0.4）
    df["hit_count_05_min"] = df[["hit_count_05", "cand_hit_count_05"]].min(axis=1) # 样本预测概率和其candidate的概率 的最小值（0.5）
    df["hit_count_sum_min"] = df[["hit_count_sum", "cand_hit_count_sum"]].min(axis=1) # 样本预测概率和其candidate的概率 的最小值（所有样本）
    
    df["hit_count_02_max"] = df[["hit_count_02", "cand_hit_count_02"]].max(axis=1) # 样本预测概率和其candidate的概率 的最大值（0.2）
    df["hit_count_03_max"] = df[["hit_count_03", "cand_hit_count_03"]].max(axis=1) # 样本预测概率和其candidate的概率 的最大值（0.3）
    df["hit_count_04_max"] = df[["hit_count_04", "cand_hit_count_04"]].max(axis=1) # 样本预测概率和其candidate的概率 的最大值（0.4）
    df["hit_count_05_max"] = df[["hit_count_05", "cand_hit_count_05"]].max(axis=1) # 样本预测概率和其candidate的概率 的最大值（0.5）
    df["hit_count_sum_max"] = df[["hit_count_sum", "cand_hit_count_sum"]].max(axis=1) # 样本预测概率和其candidate的概率 的最大值（所有样本）
    
    cossim = []
    eucdist = []
    
    eucdist1=[]
    eucdist2=[]
    eucdist3=[]
    eucdist4=[]
    eucdist5=[]
    
    chunk_size = 100_000 # 每次处理的数据量
    for i in range(0, len(df), chunk_size):  # 以chunk取数据，防止OOM 
        cossim.append(cupy.multiply(V_embed_concat[i:i+chunk_size], V_embed_concat[cand_index[i:i+chunk_size]]).sum(axis=1)) # V_embed_concat 相似度
        eucdist.append(cupy.sqrt(((V_embed_concat[i:i+chunk_size] - V_embed_concat[cand_index[i:i+chunk_size]]) ** 2).sum(axis=1))) # V_embed_concat 欧式距离 cur - cand_index
        
        eucdist1.append(cupy.sqrt(((V_embed_concat[prev_cand_index[i:i+chunk_size]] - V_embed_concat[cand_index[i:i+chunk_size]]) ** 2).sum(axis=1))) # 欧式距离1 prev_cand_index - cand_index
        eucdist2.append(cupy.sqrt(((V_embed_concat[next_cand_index[i:i+chunk_size]] - V_embed_concat[cand_index[i:i+chunk_size]]) ** 2).sum(axis=1))) # 欧式距离2 next_cand_index - cand_index
        eucdist3.append(cupy.sqrt(((V_embed_concat[i:i+chunk_size]                  - V_embed_concat[prev_cand_index[i:i+chunk_size]]) ** 2).sum(axis=1))) # 欧式距离3 cur - prev_cand_index
        eucdist4.append(cupy.sqrt(((V_embed_concat[i:i+chunk_size]                  - V_embed_concat[next_cand_index[i:i+chunk_size]]) ** 2).sum(axis=1))) # 欧式距离4 cur - next_cand_index
        eucdist5.append(cupy.sqrt(((V_embed_concat[prev_cand_index[i:i+chunk_size]] - V_embed_concat[next_cand_index[i:i+chunk_size]]) ** 2).sum(axis=1))) # 欧式距离5 prev_cand_index - next_cand_index
    
    # 将cupy数据转换为pandas数据
    df["embed_cossim"] = cupy.concatenate(cossim) 
    df["embed_eucdist"] = cupy.concatenate(eucdist)
    df["embed_eucdist1"] = cupy.concatenate(eucdist1)
    df["embed_eucdist2"] = cupy.concatenate(eucdist2)
    df["embed_eucdist3"] = cupy.concatenate(eucdist3)
    df["embed_eucdist4"] = cupy.concatenate(eucdist4)
    df["embed_eucdist5"] = cupy.concatenate(eucdist5)
    
    # 欧式距离差
    df["d0_d1"] = df["embed_eucdist"] - df["embed_eucdist1"] 
    df["d0_d2"] = df["embed_eucdist"] - df["embed_eucdist2"]
    df["d0_d3"] = df["embed_eucdist"] - df["embed_eucdist3"]
    df["d0_d4"] = df["embed_eucdist"] - df["embed_eucdist4"]
    df["d0_d5"] = df["embed_eucdist"] - df["embed_eucdist5"]
    
    for col in ["id", "name", "address", "city", "state", "zip", "country", "url", "phone", "categories", "full_address"]:
        df[f"{col}_edit_dist"] = df[col].str.edit_distance(df[col][cand_index]) # Levenshtein 距离
        # 标准化 Levenshtein 距离
        df[f"norm_{col}_edit_dist"] = df[f"{col}_edit_dist"] / df[col].str.len() # Levenshtein 距离/长度
        df[f"norm_{col}_edit_dist"] = df[f"norm_{col}_edit_dist"].replace([np.inf, -np.inf], 0) # 将inf替换为0
    
    features = [
        "diff_lon",
        "diff_lat",
        "lonlat_eucdist",
        "lonlat_manhattan",
        "lonlat_haversine_dist",
        "name_cossim",
        "full_address_cossim",
        "cat_cossim",
        "embed_cossim",
        "embed_eucdist",
        
        "embed_eucdist1",
        "embed_eucdist2",
        "embed_eucdist3",
        "embed_eucdist4",
        "embed_eucdist5",
        "d0_d1",
        "d0_d2",
        "d0_d3",
        "d0_d4",
        "d0_d5",

        "hit_count_02",
        "hit_count_03",
        "hit_count_04",
        "hit_count_05",
        "hit_count_sum",
        
        "hit_count_02_min",
        "hit_count_03_min",
        "hit_count_04_min",
        "hit_count_05_min",
        "hit_count_sum_min",
        "hit_count_02_max",
        "hit_count_03_max",
        "hit_count_04_max",
        "hit_count_05_max",
        "hit_count_sum_max"
    ]
    
    for col in ["id", "name", "address", "city", "state", "zip", "country", "url", "phone", "categories", "full_address"]:
        features.append(f"{col}_edit_dist") # 存下edit_dist类特征
        features.append(f"norm_{col}_edit_dist") # 存下标准化后的edit_dist类特征
    return df, features

In [ ]:
from collections import defaultdict

dfs = []

def inference(I, n_candiates, prefix, link):
    # 初始化预测结果
    df["hit_count_02"] = 0
    df["hit_count_03"] = 0
    df["hit_count_04"] = 0
    df["hit_count_05"] = 0
    df["hit_count_sum"] = 0
    
    df["cand_hit_count_02"] = 0
    df["cand_hit_count_03"] = 0
    df["cand_hit_count_04"] = 0
    df["cand_hit_count_05"] = 0
    df["cand_hit_count_sum"] = 0
    
    for i in range(I.shape[1]):
        print(f"Candidate rank {i} ...")
        tmp_df = df.copy() # 复制一份df
        tmp_df["match_id"] = tmp_df["id"].to_pandas().values[I[:, i]] # 第i个相似度的candidate的id

        tmp_df, features = create_features(tmp_df, i, I) # 创建特征
        tmp_df["pred"] = 0 # pred = 0
        for fold in range(CFG.n_splits):
            print(f"    ===== fold{fold} =====")
            xgb_model = ForestInference.load(
                f"../input/xgb-4ensemble-0705-v2/fs_xgb_model_{prefix}_candidate{i}_fold{fold}.json", # 加载模型
                output_class=True, # 输出为class
                model_type="xgboost_json" # 模型类型
            )

            pred = xgb_model.predict_proba(tmp_df[features].to_pandas())[:, 1] # 预测
            tmp_df["pred"] += pred / CFG.n_splits # 平均

#         dfs.append(tmp_df[tmp_df["pred"] > THRESHOLD][["id", "match_id", "pred"]])

        df["hit_count_02"] += (pred > 0.2) # 预测结果大于0.2
        df["hit_count_03"] += (pred > 0.3) # 预测结果大于0.3
        df["hit_count_04"] += (pred > 0.4) # 预测结果大于0.4 
        df["hit_count_05"] += (pred > 0.5) # 预测结果大于0.5
        df["hit_count_sum"] += pred # 预测结果
        
        ids = tmp_df.to_pandas().id.values # id
        match_ids = tmp_df.to_pandas().match_id.values # match_id
        pred_lis = tmp_df.to_pandas().pred.values # pred
        for i in range(len(tmp_df)):
            dist = 1 - pred_lis[i] # 距离 = 1 - 预测结果
            if dist > 0.9: # 如果距离大于0.9，则跳过
                continue
            link[ids[i]][match_ids[i]] = min(dist, link[ids[i]][match_ids[i]]) # 更新最小距离
        
    return link

In [ ]:
from collections import defaultdict

link = defaultdict(lambda:defaultdict(lambda: 1)) # 结果字典

link = inference(I_concat, CFG.n_candidates, "embed", link) # I_concat 训练
link = inference(I_spatial, CFG.n_spatial_candidates, "spatial", link) # I_spatial 训练

# link like:
# defaultdict(<function __main__.<lambda>()>,
#             {'E_00001118ad0191': {'E_00001118ad0191': 1.1920928955078125e-07}),
#              'E_000020eb6fed40': {'E_000020eb6fed40': 1.1920928955078125e-07}),
#              'E_00002f98667edf': {'E_00002f98667edf': 2.384185791015625e-07}),
#              'E_001b6bad66eb98': {'E_001b6bad66eb98': 1.1920928955078125e-07,
#                                   'E_0283d9f61e569d': 3.4570693969726562e-06}),
#              'E_0283d9f61e569d': {'E_0283d9f61e569d': 1.1920928955078125e-07,
#                                   'E_001b6bad66eb98': 2.7418136596679688e-06})})

In [ ]:
MAX_COST=0.4 

id_lis = []
matches = []
for source_id in tqdm(set(df.to_pandas().id)):
    tmp = []
    for key,value in link[source_id].items(): # key: match_id, value: dist
        if value <= MAX_COST: # 如果距离小于MAX_COST，则加入
            tmp.append(key)
            
    matches.append(tmp)
    id_lis.append(source_id)

In [ ]:
sub_df = pd.DataFrame()
sub_df["id"] = id_lis
sub_df["match_id"] = matches
sub_df["matches"] = sub_df["match_id"].apply(lambda x: " ".join(set(x))) # 将match_id转换为字符串

sub_df.head()

In [ ]:
sub_df.to_csv("submission.csv", index=False, columns=["id", "matches"]) # 写入文件